<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import,-setups" data-toc-modified-id="Import,-setups-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import, setups</a></span></li><li><span><a href="#Read-data" data-toc-modified-id="Read-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Read data</a></span></li><li><span><a href="#EDA" data-toc-modified-id="EDA-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>EDA</a></span><ul class="toc-item"><li><span><a href="#Impact-of-the-day-of-the-week" data-toc-modified-id="Impact-of-the-day-of-the-week-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Impact of the day of the week</a></span></li><li><span><a href="#Impact-of-the-day-of-the-month" data-toc-modified-id="Impact-of-the-day-of-the-month-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Impact of the day of the month</a></span></li><li><span><a href="#Impact-of-the-month" data-toc-modified-id="Impact-of-the-month-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Impact of the month</a></span></li><li><span><a href="#Items-categories" data-toc-modified-id="Items-categories-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Items categories</a></span></li><li><span><a href="#For-how-long-itmes-have-been-sold" data-toc-modified-id="For-how-long-itmes-have-been-sold-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>For how long itmes have been sold</a></span></li><li><span><a href="#Shops-analysis" data-toc-modified-id="Shops-analysis-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Shops analysis</a></span></li><li><span><a href="#Did-shops-differ-in-offer?" data-toc-modified-id="Did-shops-differ-in-offer?-3.7"><span class="toc-item-num">3.7&nbsp;&nbsp;</span>Did shops differ in offer?</a></span></li><li><span><a href="#Shops-location" data-toc-modified-id="Shops-location-3.8"><span class="toc-item-num">3.8&nbsp;&nbsp;</span>Shops location</a></span></li></ul></li><li><span><a href="#Decomposition-for-entire-company" data-toc-modified-id="Decomposition-for-entire-company-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Decomposition for entire company</a></span></li><li><span><a href="#Test-items-overview" data-toc-modified-id="Test-items-overview-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Test items overview</a></span></li><li><span><a href="#ARIMA" data-toc-modified-id="ARIMA-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>ARIMA</a></span></li></ul></div>

# Import, setups

In [ ]:
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import plotly.graph_objects as go
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from datetime import datetime

from statsmodels.graphics.gofplots import qqplot
from statsmodels.stats.multitest import multipletests
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.arima.model import ARIMA
import scipy

from geopy.geocoders import Nominatim

# Read data

In [ ]:
DATA_PREFIX = '../data/'

In [ ]:
item_categories = pd.read_csv(DATA_PREFIX + 'item_categories.csv')
items = pd.read_csv(DATA_PREFIX + 'items.csv')
sales_train = pd.read_csv(DATA_PREFIX + 'sales_train.csv')
shops = pd.read_csv(DATA_PREFIX + 'shops.csv')
test = pd.read_csv(DATA_PREFIX + 'test.csv')

In [ ]:
shops.head()

In [ ]:
print(len(item_categories))
item_categories.head()

In [ ]:
print(len(items))
items.head()

In [ ]:
# Merge category of item to each of the items
# items = items.merge(item_categories, on='item_category_id')
items.head()

In [ ]:
sales_train.head()

In [ ]:
sales_train = sales_train.merge(items.loc[:, ['item_id', 'item_category_id']], on='item_id')

In [ ]:
sales_train.describe()

In [ ]:
sales_train.info()

In [ ]:
sales_train.loc[:, 'date'] = pd.to_datetime(sales_train.date)

In [ ]:
sales_train = sales_train.sort_values(
    ['date', 'shop_id', 'item_category_id', 'item_id']
).reset_index(drop=True)

In [ ]:
sales_train.loc[:, 'day'] = sales_train.date.dt.day
sales_train.loc[:, 'month'] = sales_train.date.dt.month
sales_train.loc[:, 'year'] = sales_train.date.dt.year
sales_train.loc[:, 'dayofweek'] = sales_train.date.dt.dayofweek

In [ ]:
sales_train.loc[:, 'income'] = sales_train.item_price * sales_train.item_cnt_day

# EDA

In [ ]:
sales_train.head()

In [ ]:
px.line(
    sales_train.groupby('date_block_num')['item_cnt_day'].sum().reset_index(), 
    x='date_block_num', y='item_cnt_day')

In [ ]:
px.line(
    sales_train.groupby('date_block_num')['income'].sum().reset_index(), 
    x='date_block_num', y='income')

In [ ]:
months = ['', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 
          'August', 'September', 'October', 'November', 'December']
day_sales = sales_train.groupby(['year', 'month'])['item_cnt_day'].sum().reset_index()
day_sales.loc[:, 'month'] = day_sales.month.apply(lambda x: months[x])
day_sales = day_sales.reset_index()
fig = px.scatter_polar(
    day_sales, r='item_cnt_day', theta='month', color='index', 
    color_continuous_scale='viridis')
fig.update_layout(
    coloraxis_colorbar=dict(
        title="months<br>since<br>beginning",
    ),
)

## Impact of the day of the week

In [ ]:
per_day_sales = sales_train.groupby(['dayofweek', 'date'])[['item_cnt_day', 'income']].sum().reset_index()

In [ ]:
px.box(
    per_day_sales, 
    x='dayofweek', y='income', points='outliers')\
.update_layout(
    yaxis_range=[0, per_day_sales.income.quantile(0.99)]
)

In [ ]:
px.box(
    per_day_sales, 
    x='dayofweek', y='item_cnt_day', points='outliers')\
.update_layout(
    yaxis_range=[0, per_day_sales.item_cnt_day.quantile(0.99)]
)

In [ ]:
results = []
for day1 in range(7):
    for day2 in range(day1 + 1, 7):
        _, p = scipy.stats.ranksums(
            x=per_day_sales.loc[per_day_sales.dayofweek==day1, 'item_cnt_day'],
            y=per_day_sales.loc[per_day_sales.dayofweek==day2, 'item_cnt_day'],
        )
        results.append([(day1, day2), p])

In [ ]:
# Benjamini/Hochberg correction for p values when performing many tests
_, p, _, _ = multipletests([i[1] for i in results], alpha=0.05, method='fdr_bh', returnsorted=False)
for i in range(len(results)):
    results[i][1] = p[i]
    
for days, p in results:
    if p < 0.05:
        print(days)

From this test and from above histogram we see that from Monday to Thursay sales are the lowest (and also we can't say which day makes more income - those differences are statistically not significant). Every day from Friday to Sunday produces significantly more income that other working days. In particular Saturday outperforms every other day including Friday and Sunday. There is no siginificant difference between Friday and Sunday.

## Impact of the day of the month

In [ ]:
per_dayofmonth_sales = sales_train.groupby(['day', 'date'])[['item_cnt_day', 'income']].sum().reset_index()

In [ ]:
px.box(
    per_dayofmonth_sales, 
    x='day', y='income', points='outliers')\
.update_layout(
    yaxis_range=[0, per_dayofmonth_sales.income.quantile(0.99)]
)

In [ ]:
px.box(
    per_dayofmonth_sales, 
    x='day', y='item_cnt_day', points='outliers')\
.update_layout(
    yaxis_range=[0, per_dayofmonth_sales.item_cnt_day.quantile(0.99)]
)

## Impact of the month

In [ ]:
per_month_sales = sales_train.groupby(['month', 'date'])[['item_cnt_day', 'income']].sum().reset_index()

px.box(
    per_month_sales, 
    x='month', y='income', points='outliers')\
.update_layout(
    yaxis_range=[0, per_month_sales.income.quantile(0.99)]
)

In [ ]:
px.box(
    per_month_sales, 
    x='month', y='item_cnt_day', points='outliers')\
.update_layout(
    yaxis_range=[0, per_month_sales.item_cnt_day.quantile(0.99)]
)

## Items categories

In [ ]:
print(f'Number of unique items {len(items)}\nNumber of categories {len(items.item_category_id.unique())}')

In [ ]:
px.histogram(items.groupby('item_category_id')['item_id'].count(), nbins=1000)\
.update_layout(xaxis_title='group size', yaxis_title='number of groups')

In [ ]:
tmp = sales_train.groupby(['item_category_id', 'item_id'])['item_price'].mean().reset_index()
order = tmp.groupby('item_category_id').mean().reset_index().sort_values('item_price').item_category_id.tolist()
px.box(
    tmp,
    x='item_category_id', y='item_price', log_y=True, category_orders={'item_category_id': order}
)\
.update_layout(
    xaxis_type='category',
    width=1500
)

Categories differs in price to each other

In [ ]:
item_categories.loc[12] 

# indeed, the most expensive category is PS4 :)

In [ ]:
item_categories.loc[71]

# Gifts - Bags, Albums, Mouse Pads are the cheapest

## For how long itmes have been sold 

In [ ]:
sales_train

In [ ]:
tmp = pd.pivot_table(sales_train, index='item_id', values='date', aggfunc=['min', 'max'])

tmp.loc[:, 'min_date'] = tmp.loc[:, 'min'].date
tmp.loc[:, 'max_date'] = tmp.loc[:, 'max'].date
tmp = tmp.drop(['min', 'max'], axis=1)

tmp.loc[:, 'item_last'] = (tmp.loc[:, 'max_date'] - tmp.loc[:, 'min_date']).dt.days

tmp = tmp.sort_values(['min_date', 'item_last'], axis=0)

In [ ]:
px.histogram(tmp.item_last)\
.update_layout(
    title='Histogram of time of products being sold',
    xaxis_title='Number of days the product have been sold',
    yaxis_title='# of products'
)

In [ ]:
# difficult to render!
fig = go.Figure()
for i, (item, data) in enumerate(tmp.iterrows()):
    data = data.values[:2]
    fig.add_trace(
        go.Scattergl(
            x=data, y=[i, i], 
            mode='lines', 
            line=dict(color='red'),
            showlegend=False,
        )
   )
fig

del fig

In [ ]:
px.histogram(x=tmp.min_date).show()
px.histogram(x=tmp.max_date).show()
px.histogram(x=tmp.min_date.dt.month).show()
px.histogram(x=tmp.max_date.dt.month).show()
px.histogram(x=tmp.min_date.dt.day).show()
px.histogram(x=tmp.max_date.dt.day).show()

As seen from those plots - most of items are introducted within two weeks after New Year. They are recalled the most offen within first two weeks of December.

## Shops analysis

In [ ]:
len(sales_train.shop_id.unique())

In [ ]:
tmp = pd.pivot_table(sales_train, index='shop_id', values='date', aggfunc=['min', 'max'])
tmp.loc[:, 'min_date'] = tmp.loc[:, 'min'].date
tmp.loc[:, 'max_date'] = tmp.loc[:, 'max'].date
tmp = tmp.drop(['min', 'max'], axis=1)
tmp.loc[:, 'shop_last'] = (tmp.loc[:, 'max_date'] - tmp.loc[:, 'min_date']).dt.days
tmp = tmp.sort_values(['min_date', 'shop_last'], axis=0)

In [ ]:
fig = go.Figure()
for i, (item, data) in enumerate(tmp.iterrows()):
    data = data.values[:2]
    fig.add_trace(
        go.Scattergl(
            x=data, y=[i, i], 
            mode='lines', 
            line=dict(color='red'),
            showlegend=False,
        )
   )
fig.show()
del fig

On average ~50 shops were constantly active. If shop happend to be closed/open it was usually near New Year.

## Did shops differ in offer?

In [ ]:
shop_specification = pd.pivot_table(
    sales_train, index='shop_id', columns='item_category_id', values='income', aggfunc='sum'
)
sns.clustermap(shop_specification.fillna(False).astype(bool), cmap='gray', vmin=-0.5, vmax=1.2)

In [ ]:
sns.clustermap(np.nan_to_num(shop_specification, 0))

In [ ]:
sns.clustermap(np.log10(np.maximum(np.nan_to_num(shop_specification, 0), 1)))

There are 3 clusters of shops. One comprises most of the shops, they have big or huge earnings and sell similar kind of products. Within it we can distinguish approx. 6 shops that have enormous income.

The second cluster have smaller earnings and in general shops within in have smaller diversity of products.

Thist cluster is created by 2 shops which have very limited assortment. One of them sells products of completely different type than the others.

## Shops location

In [ ]:
geolocator = Nominatim(user_agent="russia cities")

In [ ]:
shops.loc[:, 'city'] = shops.shop_name.apply(lambda x: x.split()[0])
shops.loc[:, 'category'] = shops.shop_name.apply(lambda x: x.split()[1])

In [ ]:
categories_counts = shops.category.value_counts()
categories_counts

In [ ]:
idx = categories_counts[categories_counts.lt(4)].index
shops.loc[shops.category.isin(idx), 'category'] = 'other'

In [ ]:
shops.loc[shops.city == 'РостовНаДону', 'city'] = 'Ростов-На-Дону'

In [ ]:
city_pos = dict()
for city in tqdm(shops.city.unique()):
    try:
        location = geolocator.geocode(city)
        city_pos[city] = (location.latitude, location.longitude)
    except:
        continue

In [ ]:
city_pos

In [ ]:
shops.loc[:, 'latlon'] = shops.city.map(city_pos)
shops.loc[:, 'lat'] = shops.latlon.apply(lambda x: x[0])
shops.loc[:, 'lon'] = shops.latlon.apply(lambda x: x[1])
shops = shops.drop(columns='latlon')
shops.head()

Lets ask question if the product price depend on the shop and its location.

In [ ]:
mean_prices = sales_train.loc[sales_train.date_block_num==1]\
.groupby('item_id')['item_price'].mean().reset_index().rename(columns={'item_price': 'mean_item_price'})

In [ ]:
tmp = sales_train.loc[sales_train.date_block_num==1]\
.groupby(['shop_id', 'item_id'])['item_price'].mean().reset_index()
tmp = tmp.merge(mean_prices)
tmp.loc[:, 'diff'] = tmp.item_price - tmp.mean_item_price
tmp = tmp.groupby('shop_id').mean().reset_index()
tmp = tmp.merge(shops)

In [ ]:
tmp.head()

In [ ]:
fig = px.scatter_mapbox(tmp, lat="lat", lon="lon", zoom=1.8,
                        hover_name="city", color='diff')
fig.update_layout(mapbox_style="open-street-map")

# Decomposition for entire company

In [ ]:
sales_train.head()

In [ ]:
tmp = sales_train.groupby(['date_block_num', 'shop_id'])['income'].sum().reset_index()
px.line(
    tmp, x='date_block_num', y='income', color='shop_id', log_y=True, range_y=[1e5, 2e7]
).show()

tmp = sales_train.groupby(['date_block_num', 'shop_id'])['item_cnt_day'].sum().reset_index()
px.line(
    tmp, x='date_block_num', y='item_cnt_day', color='shop_id', log_y=True, range_y=[200, 20000]
).show()

In [ ]:
tmp = sales_train.groupby(['shop_id', 'date_block_num'])['item_cnt_day'].sum().reset_index()

from statsmodels.tsa.seasonal import STL

fig = plotly.subplots.make_subplots(
    rows=4, shared_xaxes=True, subplot_titles=['Observations', 'Trend', 'Season', 'Resid'],
    vertical_spacing=0.05
)
fig.update_layout(height=1000, showlegend=False)

for shop, data in tmp.groupby('shop_id'):
    stl = STL(data.item_cnt_day, period=12, seasonal=13)
    res = stl.fit()
    fig.append_trace(
        go.Scatter(
            x=data.date_block_num, y=res.observed, mode='lines', line=dict(width=1), opacity=0.3
        ), row=1, col=1
    )
    fig.append_trace(
        go.Scatter(
            x=data.date_block_num, y=res.trend, mode='lines', line=dict(width=1), opacity=0.3
        ), row=2, col=1
    )
    fig.append_trace(
        go.Scatter(
            x=data.date_block_num, y=res.seasonal, mode='lines', line=dict(width=1), opacity=0.3
        ), row=3, col=1
    )
    fig.append_trace(
        go.Scatter(
            x=data.date_block_num, y=res.resid, mode='lines', line=dict(width=1), opacity=0.3
        ), row=4, col=1
    )

tmp = sales_train.groupby('date_block_num')['item_cnt_day'].sum().reset_index() 
tmp.loc[:, 'item_cnt_day'] /= sales_train.shop_id.nunique()
stl = STL(tmp.item_cnt_day, period=3, seasonal=5, trend=5)
res = stl.fit()
fig.append_trace(
    go.Scatter(
        x=tmp.date_block_num, y=res.observed, mode='lines', line=dict(width=2, color='black'), 
    ), row=1, col=1
)
fig.append_trace(
    go.Scatter(
        x=tmp.date_block_num, y=res.trend, mode='lines', line=dict(width=2, color='black'), 
    ), row=2, col=1
)
fig.append_trace(
    go.Scatter(
        x=tmp.date_block_num, y=res.seasonal, mode='lines', line=dict(width=2, color='black'), 
    ), row=3, col=1
)
fig.append_trace(
    go.Scatter(
        x=tmp.date_block_num, y=res.resid, mode='lines', line=dict(width=2, color='black'),
    ), row=4, col=1
)

fig

In [ ]:
best_shop = sales_train.loc[sales_train.shop_id==31].groupby('date')['income'].sum()
best_shop.head()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
weekly_decompose = seasonal_decompose(best_shop, model='add', period=7)
px.line(weekly_decompose.seasonal)

In [ ]:
monthly_decompose = seasonal_decompose(best_shop - weekly_decompose.seasonal, model='add', period=30)
px.line(monthly_decompose.seasonal)

In [ ]:
annual_decompose = seasonal_decompose(best_shop - monthly_decompose.seasonal - weekly_decompose.seasonal, model='add', period=365)
px.line(annual_decompose.seasonal)

In [ ]:
px.line(best_shop - annual_decompose.seasonal - monthly_decompose.seasonal - weekly_decompose.seasonal)

# Test items overview

In [ ]:
cnt = sales_train.groupby(['shop_id', 'item_id'])['date_block_num'].nunique().reset_index()

test_covered = test.merge(cnt, how='left').fillna(0)

px.histogram(test_covered, x='date_block_num')

There is a lot of new products that have not yet been sold. That is problematic because it will require to predict  the sold amount based on similar products from the past.

In [ ]:
cnt = sales_train.groupby(['shop_id', 'item_category_id'])['date_block_num'].nunique().reset_index()

test_covered = test.merge(items.loc[:, ['item_category_id', 'item_id']]).merge(cnt, how='left').fillna(0)

px.histogram(test_covered, x='date_block_num')

# ARIMA

In [ ]:
# Useful diagnostic functions

In [ ]:
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
sales_train.head()

In [ ]:
sales_train.shop_id.value_counts().head()

In [ ]:
sales_toy = sales_train.loc[sales_train.shop_id==31]
sales_toy

In [ ]:
# sales_toy.item_id.value_counts().head()

In [ ]:
# sales_toy = sales_toy.loc[sales_toy.item_id==20949]

In [ ]:
sales_toy = sales_toy.groupby('date_block_num')[['income']].sum().reset_index()
sales_toy.head()

In [ ]:
def is_stationary(x, y, nlags=31):
    px.line(x=x, y=y).show()
    _, p, _, _ = kpss(y.dropna(), regression='c', nlags='auto')
    print(f'KPSS p-value: {p}. Data are {"NOT " if p < 0.05 else ""}stationary.')
    ac, confint, qstat, pvalues = acf(y, nlags=nlags, qstat=True, alpha=0.05, fft=False)

    fig = px.scatter(
        x=np.arange(nlags+1), 
        y=ac, 
    )
    
    fig.add_traces(
        [go.Scatter(x=np.arange(nlags + 1), y=ac - confint[:, 0],
                    fill='tozeroy', mode='none', fillcolor='rgba(219,106,100,0.3)'),
         go.Scatter(x=np.arange(nlags + 1), y=confint[:, 0] - ac,
                    fill='tozeroy', mode='none', fillcolor='rgba(219,106,100,0.3)')]
    )
    
    fig.update_layout(
        yaxis = dict(title='ACF'),
        xaxis = dict(title='lag')
    ).show()

    ac, confint = pacf(y, nlags=nlags,alpha=0.05)
    
    fig = px.scatter(
        x=np.arange(nlags+1), 
        y=ac, 
    )
    
    fig.add_traces(
        [go.Scatter(x=np.arange(nlags + 1), y=ac - confint[:, 0],
                    fill='tozeroy', mode='none', fillcolor='rgba(219,106,100,0.3)'),
         go.Scatter(x=np.arange(nlags + 1), y=confint[:, 0] - ac,
                    fill='tozeroy', mode='none', fillcolor='rgba(219,106,100,0.3)')]
    )
    
    fig.update_layout(
        yaxis = dict(title='PACF'),
        xaxis = dict(title='lag')
    ).show()

    

In [ ]:
is_stationary(sales_toy.date_block_num, sales_toy.income, nlags=len(sales_toy) // 2 - 1)

In [ ]:
# is_stationary(sales_toy.date, sales_toy.income.diff().fillna(0), nlags=380)

In [ ]:
len(sales_toy)

In [ ]:
# # for daily model
# arima = ARIMA(sales_toy.loc[:950, 'income'], order=(5, 1, 0), seasonal_order=(7, 1, 0, 365))
# res = arima.fit()

# fig = go.Figure()
# fig.add_traces(
#     [
#         go.Scatter(x=sales_toy.date, y=sales_toy.income, name='data'),
#         go.Scatter(x=sales_toy.date, y=res.predict(), name='prediction'),
#         go.Scatter(x=sales_toy.date.iloc[950:], y=res.forecast(90), name='forecast'),
#     ]
# ).show()

# res.summary()

In [ ]:
# month model
arima = ARIMA(sales_toy.loc[:28, 'income'], order=(3, 1, 0), seasonal_order=(3, 1, 0, 12))
res = arima.fit()

fig = go.Figure()
fig.add_traces(
    [
        go.Scatter(x=sales_toy.date_block_num, y=sales_toy.income, name='data'),
        go.Scatter(x=sales_toy.date_block_num, y=res.predict(), name='prediction'),
        go.Scatter(x=sales_toy.date_block_num.iloc[28:], y=res.forecast(8), name='forecast'),
    ]
).show()

res.summary()